# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
csv="cities.csv"
cities_results = pd.read_csv(csv)
cities_df=pd.DataFrame(cities_results)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ouro Preto do Oeste,45.0,BR,2020-10-27 20:05:39,40.0,-10.75,-62.22,308.19,3.16
1,Hambantota,74.0,LK,2020-10-27 20:05:40,83.0,6.12,81.12,299.13,4.19
2,College,75.0,US,2020-10-27 20:05:40,92.0,64.86,-147.80,270.15,2.58
3,Yatou,97.0,CN,2020-10-27 19:56:46,51.0,37.15,122.38,285.56,4.79
4,Devils Lake,90.0,US,2020-10-27 20:00:47,80.0,48.11,-98.87,274.15,8.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
H=cities_df["Humidity"].astype(float)
weights=H
H_mx=H.max()

In [10]:
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=H_mx,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
def k_to_c(k):
    temp_celsius = (k - 273.15) 
    return temp_celsius

In [15]:
cities_df["Max Temp"]=cities_df["Max Temp"].apply(k_to_c)

In [23]:
nc_df=cities_df.loc[(cities_df["Max Temp"]>25)&(cities_df["Max Temp"]<35)&(cities_df["Cloudiness"]>50),:]
nc_df=nc_df.dropna(how='any')
nc_df.reset_index(inplace=True)
del nc_df["index"]
nc_df.count()

City          59
Cloudiness    59
Country       59
Date          59
Humidity      59
Lat           59
Lng           59
Max Temp      59
Wind Speed    59
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
hotellist=[]
for i in range(len(nc_df)):
    lat = nc_df.loc[i]['Lat']
    lng = nc_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
nc_df["Hotel Name"] = hotellist
nc_df = nc_df.dropna(how='any')
nc_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Hambantota,74.0,LK,2020-10-27 20:05:40,83.0,6.12,81.12,25.98,4.19,Hambantota
1,San Juan del Cesar,87.0,CO,2020-10-27 20:05:40,61.0,10.77,-73.00,30.85,3.79,San Juan del Cesar
2,Kapaa,75.0,US,2020-10-27 20:05:41,83.0,22.08,-159.32,26.11,3.10,Kapaʻa
3,Faanui,99.0,PF,2020-10-27 19:57:39,82.0,-16.48,-151.75,26.61,8.51,Vaitape
4,Dhidhdhoo,71.0,MV,2020-10-27 19:57:18,76.0,6.88,73.10,27.98,3.09,Dhidhdhoo


In [55]:
hotel_df=nc_df
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Hambantota,74.0,LK,2020-10-27 20:05:40,83.0,6.12,81.12,25.98,4.19,Hambantota
1,San Juan del Cesar,87.0,CO,2020-10-27 20:05:40,61.0,10.77,-73.00,30.85,3.79,San Juan del Cesar
2,Kapaa,75.0,US,2020-10-27 20:05:41,83.0,22.08,-159.32,26.11,3.10,Kapaʻa
3,Faanui,99.0,PF,2020-10-27 19:57:39,82.0,-16.48,-151.75,26.61,8.51,Vaitape
4,Dhidhdhoo,71.0,MV,2020-10-27 19:57:18,76.0,6.88,73.10,27.98,3.09,Dhidhdhoo
5,Hithadhoo,90.0,MV,2020-10-27 20:05:45,67.0,-0.60,73.08,28.65,4.16,Addu City
6,Waslala,75.0,NI,2020-10-27 20:05:46,69.0,13.23,-85.38,26.00,1.39,Yaosca I
7,Daru,100.0,PG,2020-10-27 20:05:47,87.0,-9.08,143.21,26.08,2.81,Daru
8,Hilo,90.0,US,2020-10-27 20:05:50,74.0,19.73,-155.09,26.00,1.50,Hilo
9,Lorengau,100.0,PG,2020-10-27 20:05:59,74.0,-2.02,147.27,26.89,0.95,Lorengau


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in nc_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))